In [6]:
! pip install openpyxl gensim fasttext lightgbm multipledispatch razdel nmslib plotly

  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Attempting uninstall: pybind11
    Found existing installation: pybind11 2.9.2
ERROR: Cannot uninstall pybind11 2.9.2, RECORD file not found. You might be able to recover from this via: 'pip install --force-reinstall --no-deps pybind11==2.9.2'.


In [5]:
#папка с необходимыми файлами для обучения модели
path = "/kaggle/input/hacksai-3/"
temp = "/kaggle/working/"
#мапинг колонок из файла на колонки в коде
columns = {
    'Коды ТН ВЭД ЕАЭС':'ved_code',
    'Группа продукции':'product_group',
    'Общее наименование продукции':'product_name'
}

In [6]:
import pandas as pd
import numpy as np
import fasttext
import razdel
import os

import fasttext
from gensim.models import FastText

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, average_precision_score
from sklearn.model_selection import StratifiedShuffleSplit
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from catboost import CatBoostClassifier
import joblib

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [33]:
data = pd.read_excel(os.path.join(path,'dataset.xlsx'))
data = data.rename(columns=columns)
data = data.dropna(subset=['product_name'])
data = data[['ved_code', 'product_group', 'product_name']]

In [34]:
import re
russian_stopwords = open(os.path.join(path, 'stopwords-ru.txt'), 'r').read().split('\n')

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

def delete_punctuation(s):
    symbols = [
           '\t', '!','%','&',"'",'(',')','*','+',',','-','.', '\\', '®',
           '/', '~','«','\xad','¯','°','`','±','²','³','·','º', '»', ':',';','<','=','?','@',
           'É','Ó','Ö','×','Ø','Ü','ä','é','ö','÷','İ','Š','˂','˚','̆','Ι', 'Λ', '[','\\',']','_','`',
          '\u200e','‐','–', '—', '‘', '’', '“', '”', '•', '…', '‧', '⁰', '₂', '℃', '№', '™', 
           'Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', '↑', '−', '∞', '≤', '\uf0d2' '️','（', '）', '，', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
          ]
    
    return re.sub(r'[{}\s+]'.format(''.join(symbols)), u' ', s.replace('\xad', ' '))

def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [35]:
data['clean_product_name'] = data['product_name'].str.lower().apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))
data['product_group'] = data['product_group'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))
data = data.explode(column='product_group')
data = data[['clean_product_name', 'product_group']].dropna()
data = data[data['product_group']!='']

In [36]:
cnt_product_group = dict(data['product_group'].drop_duplicates().reset_index()[['product_group', 'index']].values)

In [37]:
data['class'] = data['product_group'].map(cnt_product_group)

In [16]:
data['class'].value_counts()

31       3932
5        2093
9        1953
219      1937
418      1854
         ... 
40213       1
16740       1
39884       1
39011       1
2040        1
Name: class, Length: 733, dtype: int64

In [38]:
data = data[data['clean_product_name']!=''].dropna()

In [39]:
train, test = train_test_split(data[['clean_product_name', 'class']], test_size=0.3, random_state=42)

Аугументация данных

In [42]:
train_big = train
def get_more_samples(data, n):
    low_sample = data["class"].value_counts().where(lambda x: x < n).dropna().index.tolist()
    low_sample = data[data["class"].isin(low_sample)]
    low_sample["clean_product_name"] = low_sample["clean_product_name"].apply(
    lambda x: ' '.join(np.random.choice(x.split(),np.random.randint(0, len(x.split())))))
    return low_sample

def get_more_samples_max(data, n):
    low_sample = data["class"].value_counts().where(lambda x: x < n).dropna().index.tolist()
    low_sample = data[data["class"].isin(low_sample)]
    low_sample["clean_product_name"] = low_sample["clean_product_name"].apply(
    lambda x: ' '.join(np.random.choice(x.split(), max(1, np.random.randint(0, len(x.split()))))))
    return low_sample

for i in range(10):
    low_sample_500 = get_more_samples(data[['clean_product_name', 'class']],50)
    low_sample_max_500 = get_more_samples_max(data[['clean_product_name', 'class']], 50)
    
    train_big = pd.concat([train_big, low_sample_500, low_sample_max_500 ])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [43]:
train.shape, train_big.shape

((52832, 2), (178512, 2))

In [45]:
fb_model_v2 = fasttext.load_model(os.path.join(path, 'fb_model_v2.bin'))

In [44]:
from sklearn.model_selection import train_test_split
import numpy as np
from lightgbm import LGBMClassifier
import joblib
from sklearn import metrics


def get_metrics(test, y_predicted):

    print('-'*10, 'product_group')
    print('micro:', metrics.f1_score(test, y_predicted, average='micro'))
    print('weighted:', metrics.f1_score(test, y_predicted, average='weighted'))

In [46]:
train, val = train_test_split(train_big[['clean_product_name', 'class']], stratify=train_big['class'], test_size=0.10, random_state=42)

In [47]:
%%time
train_vectors = np.array([fb_model_v2.get_sentence_vector(text) for text in train['clean_product_name']])
val_vectors = np.array([fb_model_v2.get_sentence_vector(text) for text in val['clean_product_name']])
test_vectors = np.array([fb_model_v2.get_sentence_vector(text) for text in test['clean_product_name']])

CPU times: user 19.4 s, sys: 397 ms, total: 19.8 s
Wall time: 19.8 s


In [48]:
lgbm_params = {
    'n_estimators': 300,
    'max_depth': 5,
    'learning_rate': 0.05,
    'n_jobs': 7,
    'random_state':42,
    'first_metric_only':True,
}

model = LGBMClassifier(**lgbm_params)
model.fit(train_vectors, train['class'], verbose=True, eval_set=(val_vectors, val['class']),  early_stopping_rounds = 15)
y_predicted = model.predict(test_vectors)
get_metrics(test['class'], y_predicted)  

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 6.21765
[2]	valid_0's multi_logloss: 11.1447
[3]	valid_0's multi_logloss: 13.1249
[4]	valid_0's multi_logloss: 15.257


KeyboardInterrupt: 

tfidf

In [49]:
import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

def evaluate_vectorizer(vectorizer, train, test):
    train_vectors = vectorizer.fit_transform(train['clean_product_name'])
    test_vectors = vectorizer.transform(test['clean_product_name'])
    joblib.dump(vectorizer, 'vectorizer.pkl')

    
    clf = LinearSVC(random_state=42)
    
    clf.fit(train_vectors, train['class'])
    joblib.dump(clf, 'model_product_group.pkl')
    
    predictions = clf.predict(test_vectors)
    get_metrics(test['class'], predictions)   

In [144]:
evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel), train, test)

---------- product_group
micro: 0.7050302521750651
weighted: 0.6799434940984934


array([  49,  439,  418, ..., 3684,   56,  285])

аугументация

In [50]:
evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel), train_big, test)

---------- product_group
micro: 0.691118668021022
weighted: 0.6794142112447626


nmslib

In [155]:
data = data.drop(['level_0', 'index',], axis=1)

In [156]:
import gensim
import nmslib
data = data.reset_index(drop=True).reset_index()

train, test = train_test_split(data[['clean_product_name', 'class', 'index']], test_size=0.20, random_state=42)
indexed_data = train#.drop_duplicates(subset = ['clean_product_name'])
indexed_data_vectors  = np.array([fb_model_v2.get_sentence_vector(text) for text in indexed_data['clean_product_name']])
test_vectors = np.array([fb_model_v2.get_sentence_vector(text) for text in test['clean_product_name']])

indexed_data_dict = dict(zip(indexed_data['index'].values, indexed_data['class'].values))
len(indexed_data_vectors), len(test_vectors)

(60380, 15095)

In [157]:
# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='napp', space='cosinesimil')
index.addDataPointBatch(data=indexed_data_vectors[:], ids=indexed_data['index'].values)
index.createIndex(print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [161]:
import nmslib
import numpy

from statistics import mode

def get_metrics(test_1):

    print('-'*10, 'product_group')
    print('micro:', metrics.f1_score(test_1['class'], test_1[0], average='micro'))
    print('weighted:', metrics.f1_score(test_1['class'], test_1[0], average='weighted'))

def get_ved_by_n(neighbours, ved_cnt, indexed_data_dict):
    neighbours = [indexed_data_dict[xi] for xi in neighbours]
    if len(set(neighbours))==1:
        return neighbours[0]
    else:
        try:
            return mode(neighbours)
        except:
            return ved_cnt[ved_cnt['index'].isin(neighbours)].head(1)['index'].tolist()[0] 

In [159]:
%%time
neighbours = index.knnQueryBatch(test_vectors, k=1, num_threads=10)
test['index'] = np.array(neighbours)[:, 0].reshape(-1)
test['distance'] = np.array(neighbours)[:, 1].reshape(-1)
test_1 = test.join(test['index'].map(indexed_data_dict).apply(pd.Series))

CPU times: user 1min 55s, sys: 63.6 ms, total: 1min 55s
Wall time: 32.3 s


In [162]:
get_metrics(test_1)

---------- product_group
micro: 0.639615766810202
weighted: 0.6345510289902021


сокращение текста

In [166]:
train, test = train_test_split(data[['clean_product_name', 'class', 'index']], test_size=0.20, random_state=42)
indexed_data = train#.drop_duplicates(subset = ['clean_product_name'])
indexed_data_vectors  = np.array([fb_model_v2.get_sentence_vector(text) for text in indexed_data['clean_product_name'].apply(lambda x: ''.join(x.split(' ')[:10]))])
test_vectors = np.array([fb_model_v2.get_sentence_vector(text) for text in test['clean_product_name'].apply(lambda x: ''.join(x.split(' ')[:10]))])

indexed_data_dict = dict(zip(indexed_data['index'].values, indexed_data['class'].values))
len(indexed_data_vectors), len(test_vectors)

(60380, 15095)

In [173]:
# initialize a new index, using a HNSW index on Cosine Similarity
index = nmslib.init(method='napp', space='cosinesimil')
index.addDataPointBatch(data=indexed_data_vectors[:], ids=indexed_data['index'].values)
index.createIndex(print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [174]:
%%time
neighbours = index.knnQueryBatch(test_vectors, k=1, num_threads=10)
test['index'] = np.array(neighbours)[:, 0].reshape(-1)
test['distance'] = np.array(neighbours)[:, 1].reshape(-1)
test_2 = test.join(test['index'].map(indexed_data_dict).apply(pd.Series))

CPU times: user 2min 2s, sys: 23.1 ms, total: 2min 2s
Wall time: 33.4 s


In [175]:
get_metrics(test_2)

---------- product_group
micro: 0.6457105001656177
weighted: 0.6401090063197566


fasstext supervised

In [190]:
data['label'] = data['class'].apply(lambda x: '__label__'+str(x))

In [191]:
train, test = train_test_split(data[['clean_product_name', 'label', 'index']], test_size=0.20, random_state=42)

In [192]:
train[['label', 'clean_product_name']].to_csv('all_text_supervised.txt', header=None, index=None)

In [194]:
test[['label', 'clean_product_name']].to_csv('test_text.txt', header=None, index=None)

In [193]:
model = fasttext.train_supervised('all_text_supervised.txt')

Read 0M words
Number of words:  29095
Number of labels: 5014
Progress: 100.0% words/sec/thread:   11904 lr:  0.000000 avg.loss:  6.335177 ETA:   0h 0m 0s


In [195]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test('test_text.txt'))

N	14479
P@1	0.492
R@1	0.492


tiny bert

In [22]:
pip install transformers sentencepiece

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

def get_embeddings(sentences):
    encoded_input = tokenizer(sentences, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
train, test = train_test_split(data[['clean_product_name', 'class']], test_size=0.3, random_state=42)
train, val = train_test_split(train[['clean_product_name', 'class']], test_size=0.10, random_state=42)

In [ ]:
%%time
train_vectors = embed_bert_cls(train['clean_product_name'].tolist(), model, tokenizer)

In [ ]:
%%time
train_vectors = np.array(get_embeddings(train['clean_product_name'].to_list()))
val_vectors = np.array(get_embeddings(val['clean_product_name'].to_list()))
test_vectors = np.array(get_embeddings(test['clean_product_name'].to_list()))

### Final model

In [11]:
import re, os,tqdm
import pandas as pd
import numpy as np
import fasttext
import razdel

from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, average_precision_score
import joblib
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

#папка с необходимыми файлами для обучения модели
path = "/kaggle/input/hacksai-3/"
temp = "/kaggle/working/"

russian_stopwords = open(os.path.join(path, 'stopwords-ru.txt'), 'r').read().split('\n')

def delete_stopwords(s):
    return ' '.join([word for word in (re.sub(r'[()\s+]', u' ', s)).split() if word.lower() not in russian_stopwords]).split()

def delete_punctuation(s):
    symbols = [
           '\t', '!','%','&',"'",'(',')','*','+',',','-','.', '\\', '®',
           '/', '~','«','\xad','¯','°','`','±','²','³','·','º', '»', ':',';','<','=','?','@',
           'É','Ó','Ö','×','Ø','Ü','ä','é','ö','÷','İ','Š','˂','˚','̆','Ι', 'Λ', '[','\\',']','_','`',
          '\u200e','‐','–', '—', '‘', '’', '“', '”', '•', '…', '‧', '⁰', '₂', '℃', '№', '™', 
           'Ⅰ', 'Ⅱ', 'Ⅲ', 'Ⅳ', '↑', '−', '∞', '≤', '\uf0d2' '️','（', '）', '，', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
          ]
    
    return re.sub(r'[{}\s+]'.format(''.join(symbols)), u' ', s.replace('\xad', ' '))

def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

In [10]:
data = pd.read_excel(os.path.join(path,'dataset.xlsx'))

#мапинг колонок из файла на колонки в коде
columns = {
    'Коды ТН ВЭД ЕАЭС':'ved_code',
    'Группа продукции':'product_group',
    'Общее наименование продукции':'product_name'
}

data = data.rename(columns=columns)
data = data.dropna(subset=['product_name'])
data = data[['ved_code', 'product_group', 'product_name']]
data['clean_product_name'] = data['product_name'].str.lower().apply(lambda x: ' '.join(delete_stopwords(delete_punctuation((x)))))
data['product_group'] = data['product_group'].fillna('').astype(str).str.split("; ").apply(lambda x: list(set(x)))
data = data.explode(column='product_group')
data = data[['clean_product_name', 'product_group']].dropna()
data = data[data['product_group']!='']

In [14]:
def tokenize_with_razdel(text):
    tokens = [token.text for token in razdel.tokenize(text)]
    
    return tokens

def evaluate_vectorizer(vectorizer, train):
    train_vectors = vectorizer.fit_transform(train['clean_product_name'])
    joblib.dump(vectorizer, 'vectorizer.pkl')

    clf = LinearSVC(random_state=42)
    
    clf.fit(train_vectors, train['product_group'])
    joblib.dump(clf, 'product_group.pkl')

In [13]:
evaluate_vectorizer(TfidfVectorizer(min_df=2, tokenizer=tokenize_with_razdel), data)